<a href="https://colab.research.google.com/github/ClouDevops101/size-limit/blob/main/recommendation-engine-udemy-ready-to-be-flasked-v7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# build a online courses recommendation system

In [1]:
# importing libraries
# src https://www.youtube.com/watch?v=ueKXSupHz6Q

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
# Download stop words
import nltk
nltk.download('stopwords')
# Saving model to file
from numpy import save
from numpy import asarray
from numpy import load

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# show available language
print(stopwords.fileids())

['arabic', 'azerbaijani', 'danish', 'dutch', 'english', 'finnish', 'french', 'german', 'greek', 'hungarian', 'indonesian', 'italian', 'kazakh', 'nepali', 'norwegian', 'portuguese', 'romanian', 'russian', 'slovene', 'spanish', 'swedish', 'tajik', 'turkish']


In [4]:
# Load the data 
#from google.colab import files
#uploaded = files.upload()

In [5]:
#Store the data 
df = pd.read_csv('udemy-data-v2.csv')
# show the first 3 rows of data 
df['title'].head(3)

0                        NoSQL Databases For Beginners
1                            Tai Chi Foundation Course
2    Music Pedagogy: 7 advices to teach a musical i...
Name: title, dtype: object

In [6]:
df.shape


(3945, 13)

In [7]:
# Creat a list of important columns for the recommendation engine
columns= ['title','description','headline','category','subcategory','instructors_name','instructors_job_title']

In [8]:
# show the data 
df[columns].head(3)


,title,description,headline,category,subcategory,instructors_name,instructors_job_title
0,NoSQL Databases For Beginners,<p>NoSQL (Non-SQL or Not-only-SQL) databases a...,Learn to Store and Retrieve Data from a Datab...,Development,Database Design & Development,Bluelime Learning Solutions,Learning made simple
1,Tai Chi Foundation Course,<p>Whether you're a beginner or experienced st...,Traditional methods. Modern teaching.,Health & Fitness,General Health,Aamir Rafi,Tai Chi practitioner & life long martial artist
2,Music Pedagogy: 7 advices to teach a musical i...,<p><strong>Are you a music teacher?</strong></...,Learn how to organize and improve your classes.,Music,Music Fundamentals,Ilse Lozoya,"University Professor, Piano Teacher, Course De..."


In [9]:
# check for missing values in the important columns
df[columns].isnull().values.any()

# data cleaning is needed

True

In [10]:
# Create a function to combine the values of the important columns into a single string
def get_important_features(data):
  important_features = []
  for i in range(0, data.shape[0]):
    important_features.append(str(data['title'][i])+' '+str(data['description'][i])+' '+str(data['headline'][i])+' '+str(data['category'][i])+' '+str(data['subcategory'][i])+' '+str(data['instructors_name'][i])+' '+str(data['instructors_job_title'][i]) )
  return important_features


In [11]:
# Create a column to hold the combined strings
df['important_features'] = get_important_features(df)

#show the data 
df.head(10)

,Course_id,title,description,headline,category,subcategory,instructors_name,instructors_job_title,price,num_subscribers,avg_rating,num_reviews,created,important_features
0,1117836,NoSQL Databases For Beginners,<p>NoSQL (Non-SQL or Not-only-SQL) databases a...,Learn to Store and Retrieve Data from a Datab...,Development,Database Design & Development,Bluelime Learning Solutions,Learning made simple,€94.99,1178,3.500000,128,2017-02-17T12:46:11Z,NoSQL Databases For Beginners <p>NoSQL (Non-SQ...
1,1117864,Tai Chi Foundation Course,<p>Whether you're a beginner or experienced st...,Traditional methods. Modern teaching.,Health & Fitness,General Health,Aamir Rafi,Tai Chi practitioner & life long martial artist,€89.99,5,0.000000,0,2017-02-17T13:28:15Z,Tai Chi Foundation Course <p>Whether you're a ...
2,1117988,Music Pedagogy: 7 advices to teach a musical i...,<p><strong>Are you a music teacher?</strong></...,Learn how to organize and improve your classes.,Music,Music Fundamentals,Ilse Lozoya,"University Professor, Piano Teacher, Course De...",€39.99,1345,4.000000,7,2017-02-17T15:50:16Z,Music Pedagogy: 7 advices to teach a musical i...
3,1118012,Ultimate Power Electronics for Electrical Powe...,"<p><strong>""Ultimate Power Electronics Course ...",Learn everything about power electronics for e...,Teaching & Academics,Engineering,Ahmed Mahdy / Khadija Academy,Electrical Power Engineer,€94.99,3186,4.592593,481,2017-02-17T16:15:02Z,Ultimate Power Electronics for Electrical Powe...
4,1118046,Facebook Advertising: Create Converting Facebo...,<p><strong>Over 8000 students have joined this...,"Learn how to select your niche, identify marke...",Marketing,Social Media Marketing,Barry North,"Award Winning ""Sexiest Voice""",€89.99,10498,4.200000,68,2017-02-17T16:48:28Z,Facebook Advertising: Create Converting Facebo...
5,1118072,Planning & Scheduling: Be the Professional fro...,<p><strong>Planning and Scheduling</strong> is...,Project Planning and Scheduling Management cou...,Business,Project Management,"Construction Project Management Guru (PSP, PMI...",Promoting Professionalism in Planning and Sche...,€109.99,3393,4.074074,615,2017-02-17T17:08:03Z,Planning & Scheduling: Be the Professional fro...
6,1118146,Wie du einen genialen Vortrag hältst,<p>Dein Vortrag ist deine Chance</p> <p>Mit d...,Mit Simon (Profi-Speaker) und Michi (NLP-Train...,Personal Development,Career Development,Simon Schnetzer & Michael Schmidt,Speaker & Persönlichkeitsentwickler,€44.99,345,4.100000,31,2017-02-17T17:50:36Z,Wie du einen genialen Vortrag hältst <p>Dein V...
7,1118268,Top Five Leadership Behaviors,<p>Every Leader Needs To Gain Their Team's Tru...,Strong Leaders Don't Just Happen: Learn These ...,Personal Development,Leadership,Donna Galatas,HR Vice Empress / CEO of The Galatas Group,€94.99,129,4.350000,27,2017-02-17T19:38:01Z,Top Five Leadership Behaviors <p>Every Leader ...
8,1118274,The Visual Guide on How Neural Networks Learn ...,<p>Course Achievements (January 2021):&nbsp;</...,The BEST Resource for Understanding Neural Net...,Development,Data Science,Mauricio Maroto,"+10,000 students and growing!",€94.99,3163,4.700000,143,2017-02-17T19:54:28Z,The Visual Guide on How Neural Networks Learn ...
9,1118300,Aprende a crear Plugins para WordPress desde c...,<p><strong>Más de 60 reseñas fantásticas !!! 4...,Es hora de saltar al siguiente paso en WordPre...,Development,Web Development,Gilbert Rodríguez,Desarrollador Web,€94.99,1457,4.500000,243,2017-02-17T20:32:46Z,Aprende a crear Plugins para WordPress desde c...


In [12]:
# creating stop word
stop_words =  stopwords.words('arabic') + stopwords.words('azerbaijani') + stopwords.words('danish') + stopwords.words('dutch') + stopwords.words('english') + stopwords.words('finnish') + stopwords.words('french') + stopwords.words('german') + stopwords.words('greek') + stopwords.words('hungarian') + stopwords.words('indonesian') + stopwords.words('italian') + stopwords.words('kazakh') + stopwords.words('nepali') + stopwords.words('norwegian') + stopwords.words('portuguese') + stopwords.words('romanian') + stopwords.words('russian') + stopwords.words('slovene') + stopwords.words('spanish') + stopwords.words('swedish') + stopwords.words('tajik') + stopwords.words('turkish')
print(stop_words)

['إذ', 'إذا', 'إذما', 'إذن', 'أف', 'أقل', 'أكثر', 'ألا', 'إلا', 'التي', 'الذي', 'الذين', 'اللاتي', 'اللائي', 'اللتان', 'اللتيا', 'اللتين', 'اللذان', 'اللذين', 'اللواتي', 'إلى', 'إليك', 'إليكم', 'إليكما', 'إليكن', 'أم', 'أما', 'أما', 'إما', 'أن', 'إن', 'إنا', 'أنا', 'أنت', 'أنتم', 'أنتما', 'أنتن', 'إنما', 'إنه', 'أنى', 'أنى', 'آه', 'آها', 'أو', 'أولاء', 'أولئك', 'أوه', 'آي', 'أي', 'أيها', 'إي', 'أين', 'أين', 'أينما', 'إيه', 'بخ', 'بس', 'بعد', 'بعض', 'بك', 'بكم', 'بكم', 'بكما', 'بكن', 'بل', 'بلى', 'بما', 'بماذا', 'بمن', 'بنا', 'به', 'بها', 'بهم', 'بهما', 'بهن', 'بي', 'بين', 'بيد', 'تلك', 'تلكم', 'تلكما', 'ته', 'تي', 'تين', 'تينك', 'ثم', 'ثمة', 'حاشا', 'حبذا', 'حتى', 'حيث', 'حيثما', 'حين', 'خلا', 'دون', 'ذا', 'ذات', 'ذاك', 'ذان', 'ذانك', 'ذلك', 'ذلكم', 'ذلكما', 'ذلكن', 'ذه', 'ذو', 'ذوا', 'ذواتا', 'ذواتي', 'ذي', 'ذين', 'ذينك', 'ريث', 'سوف', 'سوى', 'شتان', 'عدا', 'عسى', 'عل', 'على', 'عليك', 'عليه', 'عما', 'عن', 'عند', 'غير', 'فإذا', 'فإن', 'فلا', 'فمن', 'في', 'فيم', 'فيما', 'فيه', 'فيها', '

In [13]:
# convert the text to a matrix of token counts 
df['important_features'].replace('<[^<]+?>',' ',regex=True, inplace = True)
print (df['important_features'].head(3))
df['important_features'] = df['important_features'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
cm = CountVectorizer().fit_transform(df['important_features'])

# GEt the cosine similarity matrix from the count matrix
cs = cosine_similarity(cm)
#Print the cosine similarity matrix
print(cs)


0    NoSQL Databases For Beginners  NoSQL (Non-SQL ...
1    Tai Chi Foundation Course  Whether you're a be...
2    Music Pedagogy: 7 advices to teach a musical i...
Name: important_features, dtype: object
[[1.         0.08476291 0.14103213 ... 0.00295429 0.08553797 0.14755645]
 [0.08476291 1.         0.17646815 ... 0.0081325  0.10360544 0.06181151]
 [0.14103213 0.17646815 1.         ... 0.         0.14061335 0.1262184 ]
 ...
 [0.00295429 0.0081325  0.         ... 1.         0.         0.06139903]
 [0.08553797 0.10360544 0.14061335 ... 0.         1.         0.08607986]
 [0.14755645 0.06181151 0.1262184  ... 0.06139903 0.08607986 1.        ]]


In [14]:
# save data in file 
data = asarray(cs)
save('data.npy', data)


In [15]:
# Loading data from a file
cs = load('data.npy')

In [16]:
# Get the shape of the cosine similarity matrix 
cs.shape


(3945, 3945)

In [23]:
# Get the title of the course that the user likes 
title_to_search = 'برنامج مايكروسوفت أكسس 2016'

# Find the course id
course_id = df[df.title == title_to_search]['Course_id'].index
print (course_id[0])
#course_id = 251
# IndexError: index 0 is out of bounds for axis 0 with size 0
# this means that the title doesn't existe

1065


In [24]:
# Create a list of enumeration for the similarity score [(course_id, similiraty score ),(..)]
scores = list(enumerate(cs[course_id[0]]))

In [25]:
#sort the list 
sorted_scores = sorted(scores, key = lambda x:x[1], reverse  = True)
sorted_scores = sorted_scores[1:]


In [26]:
#Print the sorted scores
print(sorted_scores)

[(2872, 0.158680939997729), (1830, 0.1506983674786654), (3303, 0.1379158510305498), (2998, 0.13610420566306383), (1186, 0.12650284665650338), (2965, 0.12272562852548201), (1506, 0.11274493011541449), (3599, 0.11050442114523068), (2922, 0.10600518981422564), (2930, 0.10196337467490706), (3663, 0.09805677422563472), (3785, 0.09541069762971578), (2005, 0.0952510554470738), (2135, 0.09483485450294032), (1406, 0.09421774183428808), (265, 0.09035532193915621), (2698, 0.08612446352854669), (1311, 0.08598035917045964), (1930, 0.08580398901720908), (1647, 0.08186519189635147), (1241, 0.07835572407416602), (362, 0.07821266820029665), (2439, 0.07598793294863779), (150, 0.07539783064461299), (2172, 0.07386871344721867), (1456, 0.07041707354657985), (856, 0.06811398194651686), (3016, 0.06701505774131555), (1223, 0.06563482993646248), (2640, 0.06488705064431087), (2758, 0.06454131916624282), (2046, 0.06332916772068303), (552, 0.06285731617823549), (639, 0.06237544200908518), (1096, 0.062263901920030

In [27]:
# Create a loop to print the first 7 similar movies
j = 0
print('The 7 most recommended courses : ', title_to_search, '\nare:\n')
print('ID  Course Title' )
for item in sorted_scores:
  course_title = df[df.index == item[0]]['title'].values[0]
  course_id = df[df.index == item[0]]['Course_id'].values[0]
  #course_headline = df[df.index == item[0]]['headline'].values[0]
  num_subscribers = df[df.index == item[0]]['num_subscribers'].values[0]
  print(course_id, course_title, " | " + str(num_subscribers) )
  j=j+1
  if j>100:
    break

The 7 most recommended courses :  برنامج مايكروسوفت أكسس 2016 
are:

ID  Course Title
1217256 CSS3 و HTML تعلم تصميم المواقع من الصفر حتى الإحتراف بواسطة  | 15226
1184200 تصميم وجهات التطبيقات للجوال و المواقع (Adobe XD )  | 2233
1231750 Adobe Premiere CC 2015 in Arabic under 2 hours  | 6781
1221616 SQL Server 2016 Administration  | 2370
1163374 صمم موقع الكتروني احترافي بالووردبريس بدون اكواد فى ساعة  | 909
1220506 Microsoft Office Access 2016: Part 3 (Advanced)  | 350
1173952 AutoCAD 2017 for Mechanical Engineer  | 428
1241516 التحكم فى المتصفحات للمبتدئين  | 32
1218972 Microsoft Office Access 2016: Part 1 (Beginner)  | 340
1219224 Microsoft Office Access 2016: Part 2 (Intermediate)  | 460
1243576 Create modern MS Access forms and Advanced UI design Access  | 1350
1247516 The Complete Database Design & Modeling Beginners Tutorial  | 18249
1189512 Complete Web Development From Scratch with Yii 2 Framework  | 16442
1194352 Introduction to Microsoft Access 2016 for Absolute Beginners  |